<a href="https://colab.research.google.com/github/hwon-n/cp2/blob/hyewon/fine_tuning_bert_with_mxnet_gluon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I haven't seen a public kernel using Gluon NLP, so here is my simple example.  Adapted from https://gluon-nlp.mxnet.io/examples/sentence_embedding/bert.html

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install mxnet-cu110

     |████████████████████████████████| 325.4 MB 29 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [3]:
!pip install gluonnlp

     |████████████████████████████████| 344 kB 4.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595725 sha256=e8b5a860a6de53ae78565b34591c3a5f2c87802a5cfca53fdd02d480c500d6d7
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp


In [4]:
import warnings
warnings.filterwarnings(action='once')

import os
from pathlib import Path
import random
import multiprocessing

import numpy as np
import pandas as pd

from tqdm import tqdm_notebook, tqdm

import mxnet as mx
from mxnet import gluon
from mxnet.gluon import Block
from mxnet.gluon import nn
import gluonnlp as nlp
from gluonnlp.data import BERTTokenizer, BERTSentenceTransform

/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/local/lib/python3.7/dist-packages/mxnet/numpy/utils.py:37: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence th

In [5]:
train_tsv = '/content/drive/MyDrive/news_class9x13000/train.tsv'
test_tsv = '/content/drive/MyDrive/news_class9x13000/test.tsv'

In [6]:
train_df = pd.read_csv(train_tsv, delimiter = '\t', header = None)
train_df.columns = ['isNext', 'prompt', 'next']
train_df.head()

,isNext,prompt,next
0,1,프로크루스테스의 침대라는 그리스신화에 나오는 이야기가 있다.,이 이야기는 프로크루스테스라는 사람이 지나가는 나그네를 자신의 집으로 잡아와서 침대...
1,0,이 이야기는 프로크루스테스라는 사람이 지나가는 나그네를 자신의 집으로 잡아와서 침대...,"또, 안전행정부와 기획재정부는 전 부처와 공공기관을 대상으로 여름휴가 하루 더 가기..."
2,0,결국 침대에 사람을 맞춘다는 얘기다.,올해부터 실시하고 있는 활동보조서비스 개악 지침도 이것과 다르지 않게 잔인하다.
3,0,올해부터 실시하고 있는 활동보조서비스 개악 지침도 이것과 다르지 않게 잔인하다.,흥선대원군은 이홍장의 심문을 받은 후 바오딩(保定)으로 옮겨져 4년간 중국에서 불우...
4,1,장애인활동보조서비스는 장애로 인해 일상생활의 어려움을 겪는 사람들의 원활한 일상생활...,2007년부터 시작한 이 서비스 제도는 전국의 중증장애인들이 국가를 상대로 치열한 ...


In [7]:
test_df = pd.read_csv(test_tsv, delimiter = '\t', header = None)
test_df.columns = ['isNext', 'prompt', 'next']
test_df.head()

,isNext,prompt,next
0,1,보름달 뜨면 구름 자주 끼고,꽃이 활짝 피면 바람이 불어대지.
1,0,꽃이 활짝 피면 바람이 불어대지.,◈ 안쓰고 쌓아두면 세금...기업소득환류세제
2,1,세상일이란 모두 이런 거야.,나 홀로 웃는 까닭 아는 이 없을걸.
3,0,나 홀로 웃는 까닭 아는 이 없을걸.,권력기관에서 경력을 쌓은 인물들도 적지 않다.
4,1,정약용이 지은 독소(獨笑·홀로 웃다)라는 시이다.,이렇게 인간사 누구나 한 가지씩 걱정거리를 안고 사는 게 우리네 인생이다.


### classes

In [8]:
class BERTClassifier(Block):
    """Model for sentence (pair) classification task with BERT.

    The model feeds token ids and token type ids into BERT to get the
    pooled BERT sequence representation, then apply a Dense layer for
    classification.

    Parameters
    ----------
    bert: BERTModel
        Bidirectional encoder with transformer.
    num_classes : int, default is 2
        The number of target classes.
    dropout : float or None, default 0.0.
        Dropout probability for the bert output.
    prefix : str or None
        See document of `mx.gluon.Block`.
    params : ParameterDict or None
        See document of `mx.gluon.Block`.
    """

    def __init__(self,
                 bert,
                 num_classes=2,
                 dropout=0.0,
                 prefix=None,
                 params=None):
        super(BERTClassifier, self).__init__(prefix=prefix, params=params)
        self.bert = bert
        with self.name_scope():
            self.classifier = nn.HybridSequential(prefix=prefix)
            if dropout:
                self.classifier.add(nn.Dropout(rate=dropout))
            self.classifier.add(nn.Dense(units=num_classes))

    def forward(self, inputs, token_types, valid_length=None):  # pylint: disable=arguments-differ
        """Generate the unnormalized score for the given the input sequences.

        Parameters
        ----------
        inputs : NDArray, shape (batch_size, seq_length)
            Input words for the sequences.
        token_types : NDArray, shape (batch_size, seq_length)
            Token types for the sequences, used to indicate whether the word belongs to the
            first sentence or the second one.
        valid_length : NDArray or None, shape (batch_size)
            Valid length of the sequence. This is used to mask the padded tokens.

        Returns
        -------
        outputs : NDArray
            Shape (batch_size, num_classes)
        """
        _, pooler_out = self.bert(inputs, token_types, valid_length)
        return self.classifier(pooler_out)

In [9]:
class BERTDatasetTransform(object):
    """Dataset Transformation for BERT-style Sentence Classification or Regression.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    labels : list of int , float or None. defaults None
        List of all label ids for the classification task and regressing task.
        If labels is None, the default task is regression
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    label_dtype: int32 or float32, default float32
        label_dtype = int32 for classification task
        label_dtype = float32 for regression task
    """

    def __init__(self,
                 tokenizer,
                 max_seq_length,
                 class_labels=None,
                 pad=True,
                 pair=True,
                 has_label=True):
        self.class_labels = class_labels
        self.has_label = has_label
        self._label_dtype = 'int32' if class_labels else 'float32'
        if has_label and class_labels:
            self._label_map = {}
            for (i, label) in enumerate(class_labels):
                self._label_map[label] = i
        self._bert_xform = BERTSentenceTransform(
            tokenizer, max_seq_length, pad=pad, pair=pair)

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
          sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 3 strings:
        text_a, text_b and label.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
            label: '0'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens:  '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14
            label: 0

        For single sequences, the input is a tuple of 2 strings: text_a and label.
        Inputs:
            text_a: 'the dog is hairy .'
            label: '1'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a:  '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7
            label: 1

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 3 strings:
            (text_a, text_b, label). For single sequences, the input is a tuple
            of 2 strings: (text_a, label).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)
        np.array: classification task: label id in 'int32', shape (batch_size, 1),
            regression task: label in 'float32', shape (batch_size, 1)
        """
        if self.has_label:
            input_ids, valid_length, segment_ids = self._bert_xform(line[:-1])
            label = line[-1]
            # map to int if class labels are available
            if self.class_labels:
                label = self._label_map[label]
            label = np.array([label], dtype=self._label_dtype)
            return input_ids, valid_length, segment_ids, label
        else:
            return self._bert_xform(line)


### preprocessing

In [10]:
ctx = mx.gpu()

In [30]:
max_len = 128
pad = True
pair = True

epochs = 3
batch_size = 32
#optimizer='bertadam'
class_labels=[0,1]
lr = 1e-6
epsilon = 1e-06
warmup_ratio = 0.1
log_interval = 1000
accumulate = None

In [12]:
bert_base, vocabulary = nlp.model.get_model('bert_12_768_12', 
                                             dataset_name='wiki_multilingual_cased',
                                             pretrained=True, ctx=ctx, use_pooler=True,
                                             use_decoder=False, use_classifier=False,
                                             root='/content/drive/MyDrive/bert-mx')

In [13]:
model = BERTClassifier(bert_base, num_classes=2, dropout=0.1)
# only need to initialize the classifier layer.
model.classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
model.hybridize(static_alloc=True)

# softmax cross entropy loss for classification
loss_function = gluon.loss.SoftmaxCELoss()
loss_function.hybridize(static_alloc=True)

metric = mx.metric.Accuracy()

In [14]:
tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=False)

In [15]:
train_df['prompt'] = train_df['prompt'].astype(str)
train_df['next'] = train_df['next'].astype(str)
train_df['isNext']=train_df['isNext'].astype(int)

In [16]:
train_data_raw = train_df[['prompt', 'next', 'isNext']].values

In [17]:
pool = multiprocessing.Pool()

# transformation for data train and dev
#label_dtype = 'float32' if not task.class_labels else 'int32'
label_dtype='int32'
trans = BERTDatasetTransform(tokenizer, max_len,
                             class_labels=class_labels,
                             pad=pad, pair=pair,
                             has_label=True)

# data train
# task.dataset_train returns (segment_name, dataset)
#train_tsv = task.dataset_train()[1]
#data_train = mx.gluon.data.SimpleDataset(pool.map(trans, train_data_raw))
data_train = mx.gluon.data.SimpleDataset(train_data_raw)
data_train = data_train.transform(trans)
data_train_len = data_train.transform(
    lambda input_id, length, segment_id, label_id: length, lazy=False)
# bucket sampler for training
batchify_fn = nlp.data.batchify.Tuple(
    nlp.data.batchify.Pad(axis=0), nlp.data.batchify.Stack(),
    nlp.data.batchify.Pad(axis=0), nlp.data.batchify.Stack(label_dtype))
batch_sampler = nlp.data.sampler.FixedBucketSampler(
    data_train_len,
    batch_size=batch_size,
    #num_buckets=10,
    ratio=0,
    shuffle=True)
# data loader for training
train_data = gluon.data.DataLoader(
    dataset=data_train,
    num_workers=4,
    batch_sampler=batch_sampler,
    batchify_fn=batchify_fn)
num_train_examples = len(data_train)

/usr/local/lib/python3.7/dist-packages/gluonnlp/data/batchify/batchify.py:235: UserWarning: Padding value is not given and will be set automatically to 0 in data.batchify.Pad(). Please check whether this is intended (e.g. value of padding index in the vocabulary).
  'Padding value is not given and will be set automatically to 0 '


In [18]:
print('vocabulary used for tokenization = \n%s'%vocabulary)
print('[PAD] token id = %s'%(vocabulary['[PAD]']))
print('[CLS] token id = %s'%(vocabulary['[CLS]']))
print('[SEP] token id = %s'%(vocabulary['[SEP]']))
print('token ids = \n%s'%data_train[4][0])
print('valid length = \n%s'%data_train[4][1])
print('segment ids = \n%s'%data_train[4][2])
print('label = \n%s'%data_train[4][3])

vocabulary used for tokenization = 
Vocab(size=119547, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")
[PAD] token id = 1
[CLS] token id = 2
[SEP] token id = 3
token ids = 
[  2   0   0   0   0   0   0   0   0   0   0   0   0   0 119   3   0   0
   0   0   0   0   0   0   0   0   0   0   0   0 119   3   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1]
valid length = 
32
segment ids = 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

### training

In [19]:
optimizer = mx.optimizer.create('adam', multi_precision=True, learning_rate=lr, epsilon=epsilon)

In [20]:
all_model_params = model.collect_params()
optimizer_params = {'learning_rate': lr, 'epsilon': epsilon, 'wd': 0.01}

try:
    trainer = gluon.Trainer(all_model_params, optimizer,
                            update_on_kvstore=False)
except ValueError as e:
    print(e)
    warnings.warn(
        'AdamW optimizer is not found. Please consider upgrading to '
        'mxnet>=1.5.0. Now the original Adam optimizer is used instead.')
    trainer = gluon.Trainer(all_model_params, 'adam',
                            optimizer_params, update_on_kvstore=False)

In [31]:
step_size = batch_size * accumulate if accumulate else batch_size
num_train_steps = int(num_train_examples / step_size * epochs)
num_warmup_steps = int(num_train_steps * warmup_ratio)
step_num = 0

# Do not apply weight decay on LayerNorm and bias terms
for _, v in model.collect_params('.*beta|.*gamma|.*bias').items():
    v.wd_mult = 0.0
# Collect differentiable parameters
params = [p for p in all_model_params.values() if p.grad_req != 'null']

# Set grad_req if gradient accumulation is required
if accumulate:
    for p in params:
        p.grad_req = 'add'

#tic = time.time()
for epoch_id in range(epochs):
    metric.reset()
    step_loss = 0
    #tic = time.time()
    all_model_params.zero_grad()

    for batch_id, seqs in enumerate(train_data):
        # learning rate schedule
        if step_num < num_warmup_steps:
            new_lr = lr * step_num / num_warmup_steps
        else:
            non_warmup_steps = step_num - num_warmup_steps
            offset = non_warmup_steps / (num_train_steps - num_warmup_steps)
            new_lr = lr - offset * lr
        optimizer.set_learning_rate(new_lr)

        # forward and backward
        with mx.autograd.record():
            input_ids, valid_length, type_ids, label = seqs
            out = model(
                input_ids.as_in_context(ctx), type_ids.as_in_context(ctx),
                valid_length.astype('float32').as_in_context(ctx))
            ls = loss_function(out, label.as_in_context(ctx)).mean()
        ls.backward()

        # update
        if not accumulate or (batch_id + 1) % accumulate == 0:
            trainer.allreduce_grads()
            nlp.utils.clip_grad_global_norm(params, 1)
            trainer.update(accumulate if accumulate else 1)
            # set grad to zero for gradient accumulation
            all_model_params.zero_grad()
            step_num += 1

        step_loss += ls.asscalar()
        metric.update([label], [out])
        if (batch_id + 1) % (200) == 0:
            #log_train(batch_id, len(train_data), metric, step_loss, log_interval,
                      #epoch_id, trainer.learning_rate)
            print('[Epoch {} Batch {}/{}] loss={:.4f}, acc={:.3f}'
                  .format(epoch_id, batch_id + 1, len(train_data),
                 step_loss / log_interval, metric.get()[1]))
            step_loss = 0
    mx.nd.waitall()

[Epoch 0 Batch 200/474] loss=0.1350, acc=0.577
[Epoch 0 Batch 400/474] loss=0.1349, acc=0.577
[Epoch 1 Batch 200/474] loss=0.1348, acc=0.575
[Epoch 1 Batch 400/474] loss=0.1340, acc=0.581
[Epoch 2 Batch 200/474] loss=0.1341, acc=0.583
[Epoch 2 Batch 400/474] loss=0.1335, acc=0.589


In [32]:
model.save_parameters('j-20190616k')

### prediction

In [33]:
test_df['prompt'] = test_df['prompt'].astype(str)
test_df['next'] = test_df['next'].astype(str)
test_data_raw = test_df[['prompt', 'next']].values

In [ ]:
# batchify for data test
test_batchify_fn = nlp.data.batchify.Tuple(
    nlp.data.batchify.Pad(axis=0), nlp.data.batchify.Stack(),
    nlp.data.batchify.Pad(axis=0))

# transform for data test
test_trans = BERTDatasetTransform(tokenizer, max_len,
                                  class_labels=None,
                                  pad=pad, pair=pair,
                                  has_label=False)

data_test = mx.gluon.data.SimpleDataset(pool.map(test_trans, test_data_raw))
loader_test = mx.gluon.data.DataLoader(
    data_test,
    batch_size=batch_size,
    num_workers=4,
    shuffle=False,
    batchify_fn=test_batchify_fn)

In [ ]:
data_test[0]

In [ ]:
#value_list = []
#index_list = []
results = []
for _, seqs in enumerate(loader_test):
    input_ids, valid_length, type_ids = seqs
    out = model(input_ids.as_in_context(ctx),
                type_ids.as_in_context(ctx),
                valid_length.astype('float32').as_in_context(ctx))
    results.extend([o for o in out.asnumpy()])
    #values, indices = mx.nd.topk(out, k=1, ret_typ='both')
    #value_list.extend(values.asnumpy().reshape(-1).tolist())
    #index_list.extend(indices.asnumpy().reshape(-1).tolist())

mx.nd.waitall()



In [ ]:
results[24]

In [ ]:
predictions = [mx.nd.array(result).softmax().asnumpy()[1] for result in results]

In [ ]:
submission = pd.DataFrame.from_dict({
    'prompt': test_df['prompt'],
    'next': test_df['next'],
    'prediction': predictions
})

In [ ]:
submission.to_csv('submission.csv', index=False)